In [8]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv('../Data/replay_by_frame.csv')

In [3]:
#shouldn't be needed anymore
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df.columns

Index(['Game_ID', 'CHAR', 'Port', 'Frame', 'Pre_frame', 'Post_frame',
       'S_airborne', 'S_damage', 'S_direction', 'S_hit_stun', 'S_position_x',
       'S_position_y', 'S_shield', 'S_state', 'S_state_age', 'S_stocks',
       'B_damage', 'B_direction', 'B_joystick_x', 'B_joystick_y',
       'B_position_x', 'B_position_y', 'B_cstick_x', 'B_cstick_y', 'B_state',
       'B_raw_analog', 'B_buttons_physical', 'B_triggers_physical_l',
       'B_triggers_physical_r', 'B_triggers_logical'],
      dtype='object')

In [5]:
state_cols = ['CHAR','S_airborne', 'S_damage', 'S_direction', 'S_hit_stun', 'S_position_x',
       'S_position_y', 'S_shield', 'S_state', 'S_state_age', 'S_stocks']

In [6]:
button_cols = ['Port','B_damage', 'B_direction', 'B_joystick_x', 'B_joystick_y',
       'B_position_x', 'B_position_y', 'B_cstick_x', 'B_cstick_y', 'B_state']

In [7]:
def format_cols(input_cols, output_cols):
    cols = []
    for i in range(len(input_cols)):
        cols.append(input_cols[i] + '_P1')
        cols.append(input_cols[i] + '_P2')
    
    return ['GAME_ID', 'Frame'] + cols + output_cols

In [21]:
df.Port.unique()

array([0, 2, 1, 3], dtype=int64)

In [20]:
def format_labels(df, input_cols, output_cols, offset = 5):
    df = df.drop(['Pre_frame', 'Post_frame'], axis = 1)

In [16]:
#create labels

#TODO rewrite with reindexing and a join, should be like 1000x faster IM DUMB
#do in batches too maybe?
def format_labels(df, input_cols, output_cols, offset = 5):
    data = {}
    k = 0
    game_ids = df.Game_ID.unique()
    for game in game_ids[:2]:
        df_game = df[df.Game_ID == game]
        ports = df_game.Port.unique()
        p1 = ports[0]
        p2 = ports[1]
        for frame in tqdm(df_game.Frame.values):
            #get states of both players, designate p1 and p2
            try:
                #get current frame
                df_frame = df_game[df_game.Frame == frame]
                #get frame to predict
                df_offset = df_game[df_game.Frame == (frame + offset)]
                #get data by player for both
                df_f_p1 = df_frame[df_frame.Port == p1]
                df_f_p2 = df_frame[df_frame.Port == p2]
                df_o_p1 = df_offset[df_frame.Port == p1]
                df_o_p2 = df_offset[df_frame.Port == p2]
#                 row_1 = df_frame['GAME_ID', 'Frame'].iloc[0].values + df_f_p1[input_cols].values + df_f_p2[input_cols].values + df_o_p1[output_cols].values
#                 row_2 = df_frame['GAME_ID', 'Frame'].iloc[0].values + df_f_p1[input_cols].values + df_f_p2[input_cols].values + df_o_p2[output_cols].values
#                 #output columns for each player with player to predict
#                 data[k] = row_1
#                 k += 1
#                 data[k] = row_2
#                 k += 1
            except:
                #label not in dataset
                pass
    dataset = pd.DataFrame(data, columns = format_cols(input_cols, output_cols))
    return dataset

In [17]:
data = format_labels(df, state_cols, button_cols, 5)

  0%|          | 0/19058 [00:00<?, ?it/s]C:\Users\steph\AppData\Local\Temp/ipykernel_9528/2293080144.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_o_p1 = df_offset[df_frame.Port == p1]
 50%|████▉     | 9496/19058 [00:24<00:23, 398.85it/s]C:\Users\steph\AppData\Local\Temp/ipykernel_9528/2293080144.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_o_p2 = df_offset[df_frame.Port == p2]
100%|██████████| 4830/4830 [00:12<00:00, 391.53it/s]


In [19]:
data

,GAME_ID,Frame,CHAR_P1,CHAR_P2,S_airborne_P1,S_airborne_P2,S_damage_P1,S_damage_P2,S_direction_P1,S_direction_P2,...,Port,B_damage,B_direction,B_joystick_x,B_joystick_y,B_position_x,B_position_y,B_cstick_x,B_cstick_y,B_state
